<a href="https://colab.research.google.com/github/velmer/retrieval-information/blob/master/lab-06/Modelo_Vetorial_ri_lab_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Carregando dados

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/velmer/retrieval-information/master/lab-06/results.csv')
data.head(10)

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...
5,Rosângela uma das crianças sequestradas por f...,“Roubaram minha identidade” diz Rosângela Par...,Joana Oliveira,30/03/2019 22:21:08,Brasil,Rosângela Serra Paraná não sabe quantos anos t...,https://brasil.elpais.com/brasil/2019/03/29/po...
6,“Lógica de usar torturadores da ditadura no cr...,Aloy Jupiara coautor de 'Os porões da contrav...,Felipe Betim,01/04/2019 13:22:17,Brasil,Trajetória similar tiveram outros vários agent...,https://brasil.elpais.com/brasil/2019/03/29/po...
7,Bolsonaro troca embaixada por escritório em Je...,Autoridade Palestina convoca embaixador no Bra...,Juan Carlos Sanz,01/04/2019 12:26:32,Brasil,A devoção dos cristãos evangélicos em seu apoi...,https://brasil.elpais.com/brasil/2019/03/31/po...
8,O lado mais sombrio de Dickens,Escritor e jornalista tentou internar sua mulh...,Rafa de Miguel,10/03/2019 14:29:37,Cultura,O homem mais famoso da era vitoriana o “poeta...,https://brasil.elpais.com/brasil/2019/03/01/cu...
9,O amor entre mulheres que sacudiu o Paraguai,O drama ganhou três prêmios em Berlim e provoc...,Gregorio Belinchón,09/03/2019 12:10:23,Cultura,Depois de inclusive o da crítica internacion...,https://brasil.elpais.com/brasil/2019/03/07/cu...


# 1. Reconstruindo Índice Invertido

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê\-]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese')
tokenData = pd.DataFrame(data['text'], columns=['text'])
tokenData['tokenized_text'] = data.apply(lambda row: tknz.tokenize(row['text'].lower()), axis=1)

In [0]:
def is_tokenized_word(word):
  """
    Returns if the specified word matches all requirements
    to be a valid token.
  """
  return len(word) > 1 and any(c.isalnum() for c in word)

def build_index():
  index = {}
  num_document = 0

  for text in tokenData.tokenized_text:
    num_document += 1
    words_set = set(text)
    for word in words_set:
      if is_tokenized_word(word) and word.lower() not in stopwords:
        if word not in index.keys():
          index[word] = { 'ranking': [] }
        index[word]['ranking'].append((num_document, text.count(word)))

  return index

In [39]:
index = build_index()
indexed_data = pd.DataFrame()
indexed_data['Word'] = [word for word in index.keys()]
indexed_data['Documents (With Count)'] = [word['ranking'] for word in index.values()]
indexed_data.head(10)

,Word,Documents (With Count)
0,apurar,"[(1, 1), (214, 1), (223, 1)]"
1,instituto,"[(1, 1), (6, 1), (10, 1), (19, 1), (84, 1), (9..."
2,vesgamente,"[(1, 1)]"
3,decisão,"[(1, 3), (2, 2), (5, 2), (8, 2), (17, 1), (21,..."
4,disponível,"[(1, 1), (55, 1), (80, 1), (86, 1), (119, 1), ..."
5,passado,"[(1, 1), (3, 3), (6, 2), (8, 2), (10, 1), (16,..."
6,preconizam,"[(1, 1)]"
7,anseios,"[(1, 1), (182, 1), (198, 1), (210, 1)]"
8,promovido,"[(1, 1), (151, 1), (169, 1), (214, 1), (223, 1)]"
9,porta-voz,"[(1, 1), (5, 1), (25, 1), (64, 1), (66, 1), (7..."


# 2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário

In [0]:
def add_idf_to_index():
  M = data.text.count()
  for word in index:
    k = len(index[word]['ranking'])
    IDF = round(np.log((M+1)/k),2)
    index[word]['IDF'] = IDF

In [0]:
add_idf_to_index()

In [45]:
indexed_data['IDF'] = [word['IDF'] for word in index.values()]
indexed_data.head(10)

,Word,Documents (With Count),IDF
0,apurar,"[(1, 1), (214, 1), (223, 1)]",4.42
1,instituto,"[(1, 1), (6, 1), (10, 1), (19, 1), (84, 1), (9...",2.53
2,vesgamente,"[(1, 1)]",5.52
3,decisão,"[(1, 3), (2, 2), (5, 2), (8, 2), (17, 1), (21,...",2.02
4,disponível,"[(1, 1), (55, 1), (80, 1), (86, 1), (119, 1), ...",3.58
5,passado,"[(1, 1), (3, 3), (6, 2), (8, 2), (10, 1), (16,...",1.09
6,preconizam,"[(1, 1)]",5.52
7,anseios,"[(1, 1), (182, 1), (198, 1), (210, 1)]",4.14
8,promovido,"[(1, 1), (151, 1), (169, 1), (214, 1), (223, 1)]",3.91
9,porta-voz,"[(1, 1), (5, 1), (25, 1), (64, 1), (66, 1), (7...",2.81
